# MCXN947 Indirect chip-specific flow (using multiple commands/stages)

In the [single command example](./mcxn947_single_shot.ipynb) all necessary steps are done using a single command `el2go-host provision-device`

This notebook showcases using commands for each individual operation:

- obtain UUID from the target
- assign the target to a Device Group in EL2GO (using REST API)
- download Secure Objects from EL2GO into the target
- runs the Provisioning FW to provision the Secure Objects on the target

This flow is suitable in scenarios where the site performing the provisioning doesn't have direct connection to the EL2GO server.
In such case the flow can be broken down to 3 stages:
1) SiteA is used for UUID harvesting. All UUIDs can be placed into a single database (sqlite3 file)
    - database file will be shipped to SiteB
2) SiteB (with EL2GO connection) will download Secure Objects for all the UUIDs and store them in database
    - database file will be shipped back to SiteA
3) SiteA will provision devices using Secure Objects stored into database rather that downloading them from EL2GO

## 1. Prerequisites
- EdgeLock2GO account and API token
- MCXN947 board
- EL2GO Provisioning Firmware

## 2. Setup

### 2.1 Configuration Setup

First we'll create a configuration file containing all necessary information.
To generate a configuration template use `el2go-host get-template` command

In [1]:
# Initialization cell

from spsdk.utils.jupyter_utils import YamlDiffWidget

# setup this jupyter notebook
%env JUPYTER_SPSDK=1
%alias execute echo %l && %l
%alias_magic ! execute

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


In [2]:
YamlDiffWidget("el2go_config.diffc").html

el2go-host get-template -f mcxn947 -o config_template.yaml --force 
The EL2GO template for mcxn947 has been saved into config_template.yaml YAML file


### 2.2 HW Setup

This example uses UART for communication with the target.
You may use `nxpdevscan` utility to list COM ports with devices attached to them.

In [3]:
%! nxpdevscan --port

nxpdevscan --port 
-------- Connected NXP UART Devices --------

Port: COM5
Type: mboot device



## 3. Running individual operations

In [4]:
# Setup common variables/parameters for the commands

CONFIG = "--config config_example.yaml"
DATABASE = "--database db.sqlite3"
PORT = "--port com5"

### 3.1 Gathering UUID from the target


In [5]:
%! el2go-host get-uuid $PORT $DATABASE

el2go-host get-uuid --port com5 --database db.sqlite3 
UUID 100779615704702885679946391493172028108 stored in the database


NOTE: If there are multiple database files, they can be merged together using `el2go-host combine-uuid-db`

### 3.2 Download Secure Objects

In [6]:
%! el2go-host -v get-secure-objects $CONFIG $DATABASE

el2go-host -v get-secure-objects --config config_example.yaml --database db.sqlite3 
INFO:spsdk.el2go.database:Getting number of records without associated Secure Objects
INFO:spsdk.el2go.database:Getting UUIDs without associated Secure Objects
Found 1 UUIDs out of 1 without Secure Objects
INFO:spsdk.el2go.client:EL2G-Correlation-ID: 1adef601-9aa3-4ee2-b85e-6a6a9810d6f5
INFO:spsdk.utils.http_client:Requesting: https://api.qa.sb.edgelock2go.com/api/v1/products/340000274521/device-groups/633/devices
INFO:spsdk.utils.http_client:Response: <Response [202]>
INFO:spsdk.el2go.client:EL2G-Correlation-ID: eefc06a1-49ae-4d30-a987-18080dbf8b9e
INFO:spsdk.utils.http_client:Requesting: https://api.qa.sb.edgelock2go.com/api/v1/rtp/devices/100779615704702885679946391493172028108/secure-object-provisionings
INFO:spsdk.utils.http_client:Response: <Response [200]>
INFO:spsdk.el2go.client:EL2G-Correlation-ID: fc9e2659-a1ba-4784-98b9-4125374e79dc
INFO:spsdk.utils.http_client:Requesting: https://api.qa.sb.

NOTE: For cases when there are bigger amounts of UUIDs, there's command `el2go-host bulk-so-download` which is more performant in bulk operations comparing to downloading Secure Objects for few UUIDs.

### 3.3 Provision Secure Objects into the target

In [7]:
%! el2go-host -v provision-objects $CONFIG $DATABASE $PORT

el2go-host -v provision-objects --config config_example.yaml --database db.sqlite3 --port com5 
INFO:spsdk.mboot.mcuboot:Connect: identifier='uart', device=com5
INFO:spsdk.mboot.mcuboot:CMD: GetProperty(UniqueDeviceIdent, index=0)
INFO:spsdk.mboot.mcuboot:CMD: Status: 0 (0x0) Success.
INFO:spsdk.mboot.mcuboot:Closing: identifier='uart', device=com5
INFO:spsdk.el2go.database:Getting Secure Objects for UUID: 100779615704702885679946391493172028108
INFO:spsdk.mboot.mcuboot:Connect: identifier='uart', device=com5
Writing User config data to: 0x20000000
INFO:spsdk.mboot.mcuboot:CMD: WriteMemory(address=0x20000000, length=32, mem_id=0)
INFO:spsdk.mboot.mcuboot:CMD: GetProperty(MaxPacketSize, index=0)
INFO:spsdk.mboot.mcuboot:CMD: Status: 0 (0x0) Success.
INFO:spsdk.mboot.mcuboot:CMD: Max Packet Size = 256
INFO:spsdk.mboot.mcuboot:CMD: Status: 0 (0x0) Success.
INFO:spsdk.mboot.mcuboot:CMD: Successfully Send 32 out of 32 Bytes
Writing Secure Objects to: 0x20000100
INFO:spsdk.mboot.mcuboot:CMD:

## 4. Quick verification

A full provisioning verification is not possible without a dedicated user application that would test all the Secure Objects. 
Test of RKTH and image encryption keys is done indirectly by loading the user SB3.1 file.

For demonstration purposes, we can read out Secure Object's metadata stored in flash.
The address is defined in the config file as `secure_objects_address` (in our case it's 0x1000).

Each Secure Objects starts with: b"\x40\x0Bedgelock2go"

In [8]:
%! blhost --port com5 read-memory 0x1000 0x10 --use-hexdump

blhost --port com5 read-memory 0x1000 0x10 --use-hexdump 
Reading memory
00000000: 40 0B 65 64 67 65 6C 6F  63 6B 32 67 6F 41 04 00  @.edgelock2goA..
Response status = 0 (0x0) Success.
Response word 1 = 16 (0x10)
Read 16 of 16 bytes.
